# Gridded Analysis

Analyses of data availability at the country level are limited in terms of spatial resolution. The spatial distribution of OSM contributions is highly heterogeneous and varies inside a given country. Here, we perform a gridded analysis of data availability based on 1 km cells.

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-of-interest" data-toc-modified-id="Data-of-interest-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data of interest</a></span></li><li><span><a href="#Create-a-100-km-grid-over-Africa" data-toc-modified-id="Create-a-100-km-grid-over-Africa-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create a 100 km grid over Africa</a></span></li></ul></div>

## Data of interest

OSM properties of interest for the road network:

* `highway`: type of road ;
* `maxspeed`: max. speed of road segment ;
* `width`: width of the road ;
* `surface`: surface of the road ;
* `lanes`: number of lanes.

OSM properties of interest not related to the road network:

* `landuse`: polygons describing land use ;
* `natural`: polygons describing a natural land cover ;
* `leisure`: polygons identifying a leisure object ;
* total number of nodes?

The objective is to compute multi-temporal (each 3 months between 2010 and 2019) values for each 1 km cell:

* km of roads for each road type ;
* km of roads with a `surface` property ;
* km of roads with a `maxspeed` property ;
* km of roads with a `width` property ;
* km of roads with a `lanes` property ;
* total number of nodes ;
* number of landuse objects ;
* number of leisure objects ;
* number of points of interest.

## Create a 100 km grid over Africa

In [12]:
%matplotlib inline

import os
from itertools import product
import subprocess

import numpy as np
import matplotlib.pyplot as plt
import geojson
from fiona import crs
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import osmium

In [13]:
input_dir = os.path.abspath('../data/input')
output_dir = os.path.abspath('../data/output')

In [2]:
WGS84 = crs.from_epsg(4326)

# Use a better CRS for Africa
AFRICA_EQUAL_AREA = crs.from_string(
    "+proj=aea +lat_1=20 +lat_2=-23 +lat_0=0 +lon_0=25 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m no_defs"
)

In [3]:
countries = gpd.read_file('../data/input/countries.geojson')
countries.crs = WGS84
countries.to_crs(AFRICA_EQUAL_AREA, inplace=True)

In [4]:
def grid_dimensions(bounds, cell_size):
    """Get grid height and width."""
    xmin, ymin, xmax, ymax = bounds
    height = (ymax - ymin) / cell_size
    width = (xmax - xmin) / cell_size
    return height, width

def round_bounds(bounds, cell_size):
    """Round xmax and ymax so that height and width are integers."""
    height, width = grid_dimensions(bounds, cell_size)
    x_delta = np.ceil(width) - width
    y_delta = np.ceil(height) - height
    xmin, ymin, xmax, ymax = bounds
    xmax += x_delta * cell_size
    ymax += y_delta * cell_size
    return (xmin, ymin, xmax, ymax)

In [5]:
CELL_SIZE = 100000
bounds = round_bounds(countries.total_bounds.round(6), CELL_SIZE)
height, width = grid_dimensions(bounds, CELL_SIZE)

In [6]:
def get_cell_size(bounds, height, width):
    """Get cell size from grid bounds, height and width."""
    xmin, ymin, xmax, ymax = bounds
    xres = (xmax - xmin) / width
    yres = (ymax - ymin) / height
    return xres, yres

def create_grid(bounds, height, width):
    """Create grid from bounds, height and width. Returns a list
    of polygons.
    """
    assert round(height) == height
    assert round(width) == width
    xres, yres = get_cell_size(bounds, height, width)
    xmin, ymin, xmax, ymax = bounds
    polygons = []
    for x in np.arange(xmin, xmax, xres):
        for y in np.arange(ymin, ymax, yres):
            polygons.append(Polygon([
                (x, y),
                (x + xres, y),
                (x + xres, y - yres),
                (x, y - yres)
            ]))
    return polygons

In [7]:
polygons = create_grid(bounds, height, width)
grid = gpd.GeoSeries(polygons)
grid.crs = AFRICA_EQUAL_AREA

In [8]:
# Get rid of cells that does not intersect land
africa = countries.unary_union
grid = grid[grid.intersects(africa)]

In [14]:
grid.to_file(os.path.join(input_dir, 'grid.gpkg'), driver='GPKG')

In [9]:
len(grid), width, height

(3207, 81.0, 92.0)